In [88]:
import os
import pandas as pd
import numpy as np
import re
import datetime
import pickle
import json
import time
from nltk.tokenize import sent_tokenize
import nltk
from collections import Counter

from nltk.stem import WordNetLemmatizer,PorterStemmer
  
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

os.chdir('..')
print(os.getcwd())
from utils import get_fulltext,get_fname,fulltext_exists
os.chdir('./data_processing/')

/Users/yiweiluo/scientific-debates


In [2]:
df = pd.read_pickle('../data_scraping/dedup_combined_df.pkl')
df.head(10)

,url,title,date,domain,stance,topic,is_AP,year,pretty_domain
0,www.buzzfeednews.com/article/tasneemnashrulla/...,"""eat the babies"" viral video at aoc town hall ...",2019-10-04 00:00:00,buzzfeed,pro,cc,None,2019.0,Buzzfeed
1,www.buzzfeednews.com/article/passantino/extrem...,"""extremely likely"" global warming is man-made,...",2013-09-27 00:00:00,buzzfeed,pro,cc,None,2013.0,Buzzfeed
2,shotofprevention.com/2010/11/03/history-makes-...,"""history"" makes headlines with launch of new w...",2020-03-13 14:32:02,https://shotofprevention/,pro,vax,False,2020.0,Shot of Prevention
3,www.buzzfeednews.com/article/andrewkaczynski/i...,"""it's global warming, stupid"" - buzzfeed news",2012-11-01 00:00:00,buzzfeed,pro,cc,None,2012.0,Buzzfeed
4,www.buzzfeednews.com/article/tasneemnashrulla/...,"""japan dropped an atomic bomb on america durin...",2014-02-24 00:00:00,buzzfeed,pro,cc,None,2014.0,Buzzfeed
5,www.buzzfeednews.com/article/llevin/opinion-im...,"""look at my record, child"": joe biden showed m...",2019-10-31 00:00:00,buzzfeed,pro,cc,None,2019.0,Buzzfeed
6,www.breitbart.com/politics/2019/06/14/orourke-...,"""president o'rourke will end oil and gas lease...",2019-06-14 00:00:00,breitbart,anti,cc,None,2019.0,Breitbart
7,www.buzzfeednews.com/article/andrewkaczynski/s...,smoking doesnt kill and other great old opeds ...,2015-03-31 00:00:00,buzzfeed,pro,cc,None,2015.0,Buzzfeed
8,www.foxnews.com/world/100-carbon-tax-by-2030-c...,"$100 carbon tax by 2030 could save climate, sa...",2017-05-29 00:00:00,fox,anti,cc,None,2017.0,Fox
21585,childrenshealthdefense.org/news/4-billion-and-...,$4 billion and growing: u.s. payouts for vacc...,2018-11-19 00:00:00,chd,anti,vax,None,2018.0,Children's Health Defense


# Contents:
* [Define functions](#Define-functions)
* [Extract clausal complements with SpaCy](#Extract-embedded-clauses-with-SpaCy)

# Define functions

We add a neural coref resolution step to the default SpaCy pipeline: https://github.com/huggingface/neuralcoref

In [9]:
import spacy
nlp = spacy.load('en_core_web_sm')

# First way we can control a parameter
#neuralcoref.add_to_pipe(nlp, greedyness=0.75)
import neuralcoref
neuralcoref.add_to_pipe(nlp)

Helper functions for the main ```get_quotes``` function:

In [4]:
def is_good_verb_dep(dep):
    return dep[:3] == 'aux' or dep[:3] == 'adv' or dep == 'det' or dep == 'rel' or dep == 'prep' or dep[-3:] == 'obj' or dep[-3:] == 'mod' or dep == 'prt' or (dep[-4:] == 'comp' and dep != 'ccomp')

def is_good_subj_dep(dep):
    return dep != 'ccomp'

def is_ROOT(tok):
    return tok.dep_ == 'ROOT' or tok.dep_[-2:] == 'cl' or tok.dep_ == 'ccomp' or \
            (tok.dep_ == 'conj' and tok.head.dep_ == 'ROOT')

REL_PRONOUNS = set(['who', 'whom', 'whose', 'which', 'that'])
def is_rel_pronoun(tok):
    tok = tok.lower().strip()
    return tok in REL_PRONOUNS

In [208]:
def get_quotes(text):
    # Step 0. Run pipeline.
    doc = nlp(text)

    # Step 1. Figure out which tokens to tag w/ coreferring token
    to_annotate = {}
    for clust in doc._.coref_clusters:
        non_main_mentions = [x for x in clust.mentions if x != clust.main]
        for mention in non_main_mentions:
            start = mention.start
            to_annotate[start] = True
            end = mention.end
            if start+1 != end:
                for ix in range(start+1,end):
                    to_annotate[ix] = False
    to_annotate.update({ix: False for ix in range(len(doc)) if ix not in to_annotate})

    annotated_tokens = {}
    for token in doc:
        #print(token.i,token.text,token.head,token.dep_)
        if to_annotate[token.i]:
            #print('Main span:',token._.coref_clusters[0].main)
            # print('Main span start token_ix, end token_ix:',token._.coref_clusters[0].main.start,
            #      token._.coref_clusters[0].main.end)
            # print('\n')
            annotated_tokens[token.i] = token._.coref_clusters[0].main
        else:
            annotated_tokens[token.i] = None

    # Step 3. Go through each sentence in the doc, extract up to 1 quote from each sentence
    quote_objs = []
    for sent in doc.sents:
        #print(sent)

        VERBS = list(np.unique([token.head for token in sent if token.dep_ == 'ccomp']))
        #print(VERBS)

        # Extract everything else for each VERB
        for VERB in VERBS:
            #print("\nCcomp dependency found! For quoting verb '{}'".format(VERB))
            # Extract the rest of the quoting verb
            verb_deps = [x for x in VERB.children if is_good_verb_dep(x.dep_)]
            #print("\tFound children:",verb_deps)
            for x in verb_deps:
                new_children = [c for c in x.children if is_good_verb_dep(c.dep_)]
                #print("\tAdding children of {}:".format(x.text),new_children)
                verb_deps.extend(new_children)
                #print("\tUpdated verb deps:",verb_deps)

            # If verb's head is not itself (i.e., verb is not the ROOT),
            # recursively trace back to ROOT, then add all children of ROOT
            ROOT = VERB
            if is_ROOT(ROOT):
                pass
                #print("\n\tVerb '{}' is the ROOT.".format(ROOT))
                #verb_deps.append(ROOT)
            else:
                #print('\n\tFinding ROOT...')
                while not is_ROOT(ROOT):
                    ROOT = ROOT.head
                    #print('\t\tCurrent root:',ROOT)
                assert is_ROOT(ROOT)
                verb_deps.append(ROOT)

                #print("\tAdding children of ROOT...")
                root_deps = [x for x in ROOT.children if is_good_verb_dep(x.dep_)]
                #print("\tFound ROOT deps:",root_deps)
                for x in root_deps:
                    new_children = [c for c in x.children if is_good_verb_dep(c.dep_)]
                    #print("\tAdding children of {}:".format(x.text),new_children)
                    root_deps.extend(new_children)
                    #print("\tUpdated ROOT deps:",root_deps)

                #print("\tAdding ROOT deps to verb deps...")
                verb_deps.extend([x for x in root_deps if x != VERB and x not in verb_deps])
                #print("\tUpdated verb deps:",verb_deps)

            NEG,IS_NEG,neg_children = None,None,None
            SUBJECT,subj_children = None,None

            #print("\nLooking for SUBJECT and NEGATION(s)...")
            for child in ROOT.children:
                if child.dep_[:5] == 'nsubj' or child.dep_ == 'expl':
                    SUBJECT = child
                    #print("\tFound SUBJECT:",SUBJECT)
                    if SUBJECT.head.dep_ == 'relcl' and is_rel_pronoun(SUBJECT.text): # we're dealing with the subject of a rel clause
                        #print("\tFound quote inside a relative clause. Finding antecedent subject...")
                        SUBJECT = SUBJECT.head.head
                        #print("\tTrue subject:",SUBJECT)
                    #print("Subject token '{}' is in a coref cluster:".format(SUBJECT),SUBJECT._.in_coref)

                if child.dep_[:3] == 'neg':
                    NEG = child
                    verb_deps.append(NEG)
                    neg_children = [c for c in NEG.children if c != VERB]
                    #print("n\tAdding new NEG children:",neg_children)
                    for x in neg_children:
                        new_children = [c for c in x.children]
                        #print("\tNew NEG grandchildren:",new_children)
                        neg_children.extend(new_children)
                        #print("\tUpdated neg_children:",neg_children)
                    verb_deps.extend(neg_children)
                    #print("\tUpdated verb_deps:",verb_deps)
                    IS_NEG = VERB in NEG.head.children or VERB == NEG.head

            if SUBJECT is None and (ROOT.dep_ == 'acl' or ROOT.dep_ == 'advcl'):
                main_verb = ROOT.head
                #print([(c.text,c.dep_) for c in main_verb.children])
                SUBJS = [c for c in main_verb.children if c.dep_[:5] == 'nsubj' or c.dep_ == 'expl']
                SUBJECT = SUBJS[0] if len(SUBJS) > 0 else None

            # Get rest of subject tokens
            if SUBJECT is not None:
                #print("\nFound SUBJECT:",SUBJECT)
                #print("\tAdding children of SUBJECT...")
                subj_children = [c for c in SUBJECT.children if is_good_subj_dep(c.dep_)]
                #print("\tFound children:",subj_children)
                for x in subj_children:
                    new_children = [c for c in x.children if is_good_subj_dep(c.dep_)]
                    #print("\tAdding children of child {}:".format(x.text),new_children)
                    subj_children.extend(new_children)
                    #print("\tUpdated subject children:",subj_children)

            #print("\nFinding quote introduced by '{}'...".format(VERB))
            emb_main_verbs = [c for c in VERB.children if c.dep_ == 'ccomp']
            #print("\tMain verbs of embedded clause:",emb_main_verbs)
            #assert len(emb_main_verbs) <= 2
            for emb_main_verb in emb_main_verbs:
                #print("\tMain *verb*:",emb_main_verb)

                # Recursively get all children of main verb of embedded clause
                #print("\nRecursively getting children of main verb of embedded clause...\n")
                #print("*"*50)
                children_queue = [x for x in emb_main_verb.children]
                #print("\tAdding children of main verb:",children_queue)
                for x in children_queue:
                    new_children = [c for c in x.children]
                    #print("\tAdding children of {}:".format(x.text),new_children)
                    children_queue.extend(new_children)
                    #print("\tNew children queue:",children_queue)

                # Sort children and matrix verb to be in correct order
                # Replace w/ coreferring text as necessary
                children_and_indices_and_coref = [(c,c.i,annotated_tokens[c.i]) for c in children_queue+[emb_main_verb]]
                sorted_ = sorted(children_and_indices_and_coref,key=lambda x:x[1])
                sorted_verb_tokens = sorted([(c,c.i,annotated_tokens[c.i]) for c in verb_deps+[VERB]],key=lambda x:x[1])
                #print("\n\tSorted verb tokens:",sorted_verb_tokens)
                if SUBJECT is not None:
                    sorted_subj_tokens = sorted([(c,c.i,annotated_tokens[c.i]) for c in subj_children+[SUBJECT]],key=lambda x:x[1])
                    #print("\tSorted subject tokens:",sorted_subj_tokens)
                else:
                    sorted_subj_tokens = None
                if NEG is not None:
                    sorted_neg_tokens = sorted([(c,c.i,annotated_tokens[c.i]) for c in neg_children+[NEG]],key=lambda x:x[1])
                else:
                    sorted_neg_tokens = None

                dict_ = {'quote tokens':[tup[0] for tup in sorted_],
                        'quote tokens coref':[tup[0] if tup[-1] is None else tup[-1] for tup in sorted_],
                        'verb tokens':[tup[0] for tup in sorted_verb_tokens],
                        'verb tokens coref':[tup[0] if tup[-1] is None else tup[-1] for tup in sorted_verb_tokens],
                        'main verb':VERB,
                        'main verb coref':annotated_tokens[VERB.i] if annotated_tokens[VERB.i] else VERB,
                       'subject tokens':[tup[0] for tup in sorted_subj_tokens] if sorted_subj_tokens is not None else None,
                        'subject tokens coref':([tup[0] if tup[-1] is None else tup[-1] for tup in sorted_subj_tokens])
                                   if sorted_subj_tokens is not None else None,
                       'main subject':SUBJECT if SUBJECT is not None else None,
                        'main subject coref':annotated_tokens[SUBJECT.i] if SUBJECT is not None and annotated_tokens[SUBJECT.i] else
                                   (SUBJECT if SUBJECT is not None else None),
                       'neg tokens':[tup[0] for tup in sorted_neg_tokens] if sorted_neg_tokens is not None else None,
                       'main neg':NEG if NEG is not None else None,
                       'is neg':IS_NEG}

                res_ = {}
                res_['quote lemmas'] = [x.lemma_ for x in dict_['quote tokens']]
                res_['quote lemmas coref'] = [x.lemma_ for x in dict_['quote tokens coref']]
                res_['verb lemmas'] = [x.lemma_ for x in dict_['verb tokens']]
                res_['verb lemmas coref'] = [x.lemma_ for x in dict_['verb tokens coref']]
                res_['main verb lemma'] = dict_['main verb'].lemma_
                res_['main verb lemma coref'] = dict_['main verb coref'].lemma_
                res_['subject lemmas'] = [x.lemma_ for x in dict_['subject tokens']] if dict_['subject tokens'] is not None else None
                res_['subject lemmas coref'] = [x.lemma_ for x in dict_['subject tokens coref']] if dict_['subject tokens coref'] is not None else None
                res_['main subject lemma'] = dict_['main subject'].lemma_ if dict_['main subject'] is not None else None
                res_['main subject lemma coref'] = dict_['main subject coref'].lemma_ if dict_['main subject coref'] is not None else None
                res_['neg lemmas'] = [x.lemma_ for x in dict_['neg tokens']] if dict_['neg tokens'] is not None else None
                res_['main neg lemma'] = dict_['main neg'].norm_ if dict_['main neg'] is not None else None
                res_['quote text'] = [x.text for x in dict_['quote tokens']]
                res_['verb text'] = [x.text for x in dict_['verb tokens']]
                res_['main verb text'] = dict_['main verb'].text
                res_['subject text'] = [x.text for x in dict_['subject tokens']] if dict_['subject tokens'] is not None else None
                res_['main subject text'] = dict_['main subject'].text if dict_['main subject'] is not None else None
                res_['is neg'] = dict_['is neg']

                quote_objs.append(res_)

    return quote_objs

# Extract clausal complements with SpaCy

(Did this part on the cluster.)

In [209]:
s = time.time()

In [210]:
time.time()-s

0.5713517665863037

## First 100 URLs

In [232]:
start_time = time.time()
for url_ix in range(9000,10000):
    curr_url = df.url.values[url_ix]
    quotes = get_quotes(get_fulltext(curr_url)[0])
    fname = get_fname(curr_url)
    with open('./extracted_quotes/{}.jl'.format(fname),'w+') as f:
        for res in quotes:
            json.dump(res, f)
            f.write('\n')
    if url_ix % 100 == 0:
        print('{}\t{}\tElapsed time in seconds:'.format(url_ix,fname),(time.time()-start_time)/60.)

9000	www.motherjones.com[SEP]environment[SEP]2018[SEP]10[SEP]whats-really-needed-to-fight-climate-change-is-money-and-trump-will-not-contribute-a-cent[SEP]	Elapsed time in seconds: 0.08422701358795166
9100	www.thenation.com[SEP]article[SEP]archive[SEP]why-are-americans-switching-to-renewable-energy-because-its-actually-cheaper[SEP]	Elapsed time in seconds: 2.9183404127756756
9200	www.washingtonpost.com[SEP]news[SEP]wonk[SEP]wp[SEP]2014[SEP]05[SEP]12[SEP]why-society-is-failing-to-stop-global-warming-in-one-90-second-video[SEP]	Elapsed time in seconds: 6.324792118867238
9300	www.motherjones.com[SEP]politics[SEP]2014[SEP]09[SEP]heres-deal-big-un-climate-summit-next-week[SEP]	Elapsed time in seconds: 9.614413384596507
9400	dailycaller.com[SEP]2015[SEP]10[SEP]21[SEP]a-climate-treaty-is-pointless-unless-china-and-india-cut-co2[SEP]	Elapsed time in seconds: 12.190600180625916
9500	www.activistpost.com[SEP]2012[SEP]09[SEP]is-cdcs-mandated-vaccine-schedule_29.html	Elapsed time in seconds: 17.79

In [231]:
len(os.listdir('extracted_quotes'))

9000

# Retrieval of extracted complement clauses from filename

In [322]:
CLUSTER_DIR = '/Users/yiweiluo/Desktop/extracted_quotes/'
LOCAL_DIR = './extracted_quotes/'

cluster_quotes = os.listdir(CLUSTER_DIR)
local_quotes = os.listdir(LOCAL_DIR)

In [323]:
len(local_quotes),len(cluster_quotes)

(17621, 80)

In [331]:
17621+4782

22403

In [328]:
set(local_quotes).intersection(set(cluster_quotes))

set()

In [329]:
len(set(local_quotes) | set(cluster_quotes))

17701

In [307]:
for q in cluster_quotes:
    fname = '{}.jl'.format(get_fname(df.url.values[int(q.split('_')[-1].split('.jsonlist')[0])]))
    #print(fname)
    #print(LOCAL_DIR+fname)
    os.rename(CLUSTER_DIR+q,LOCAL_DIR+fname)

In [305]:
os.getcwd()

'/Users/yiweiluo/scientific-debates/data_processing'

In [299]:
cluster_quotes[:5]

['url_no_10000.jsonlist',
 'url_no_10001.jsonlist',
 'url_no_10002.jsonlist',
 'url_no_10003.jsonlist',
 'url_no_10004.jsonlist']

In [303]:
for q in cluster_quotes[:5]:
    fname = '{}.jl'.format(get_fname(df.url.values[int(q.split('_')[-1].split('.jsonlist')[0])]))
    print(fname)
    print(fname.split('.jl'[0]) in named_local_quotes)
    print(LOCAL_DIR+fname)

grist.org[SEP]news[SEP]big-polluters-tell-supreme-court-theyre-worried-for-chinese-restaurateurs[SEP]?utm_source=syndication&utm_medium=rss&utm_campaign=feed.jl
False
./extracted_quotes/grist.org[SEP]news[SEP]big-polluters-tell-supreme-court-theyre-worried-for-chinese-restaurateurs[SEP]?utm_source=syndication&utm_medium=rss&utm_campaign=feed.jl
www.theguardian.com[SEP]environment[SEP]2008[SEP]jun[SEP]17[SEP]forests.endangeredhabitats.jl
False
./extracted_quotes/www.theguardian.com[SEP]environment[SEP]2008[SEP]jun[SEP]17[SEP]forests.endangeredhabitats.jl
www.csmonitor.com[SEP]Science[SEP]Discoveries[SEP]2009[SEP]1010[SEP]biggest-news-youve-never-heard-earth-isnt-warming.jl
False
./extracted_quotes/www.csmonitor.com[SEP]Science[SEP]Discoveries[SEP]2009[SEP]1010[SEP]biggest-news-youve-never-heard-earth-isnt-warming.jl
www.theguardian.com[SEP]technology[SEP]blog[SEP]2013[SEP]nov[SEP]13[SEP]bike-future-technology-2013.jl
False
./extracted_quotes/www.theguardian.com[SEP]technology[SEP]blog[S

In [311]:
QUOTES_DIR = './extracted_quotes/'

In [312]:
extracted_set = set(os.listdir(QUOTES_DIR))
len(extracted_set)

17621

In [313]:
for ix in range(0,17621):
    url = df.url.values[ix]
    fname = get_fname(url)
    assert '{}.jl'.format(fname) in extracted_set

In [314]:
extracted_set

{'dailycaller.com[SEP]2016[SEP]08[SEP]04[SEP]hillary-considers-axing-ethanol-mandate-after-writing-an-oped-supporting-it[SEP].jl',
 'hotair.com[SEP]archives[SEP]2012[SEP]07[SEP]03[SEP]bam-trend-of-co2-emissions-falling-toward-1991-levels[SEP].jl',
 'www.buzzfeednews.com[SEP]article[SEP]matthewzeitlin[SEP]sodastream-to-shut-down-controversial-west-bank-plant.jl',
 'www.guardian.co.uk[SEP]theguardian[SEP]2010[SEP]oct[SEP]16[SEP]stoke-pottery-industry-china-ian-jack.jl',
 'feedproxy.google.com[SEP]~r[SEP]breitbart[SEP]~3[SEP]dDbCobm-wM4[SEP].jl',
 'www.vox.com[SEP]2015[SEP]7[SEP]24[SEP]9035803[SEP]fossil-fuel-companies-cost-of-carbon.jl',
 'www.vox.com[SEP]energy-and-environment[SEP]2018[SEP]12[SEP]5[SEP]18126920[SEP]xcel-energy-100-percent-clean-carbon-free.jl',
 'www.activistpost.com[SEP]2016[SEP]07[SEP]is-the-cia-manipulating-the-weather.html.jl',
 'www.voicesforvaccines.org[SEP]what-hpv-can-do[SEP].jl',
 'www.alternet.org[SEP]environment[SEP]hillary-clinton-tried-push-fracking-other-n

In [318]:
def get_quotes_from_file(url_ix,df,missing):
    url = df.url.values[url_ix]
    fname = '{}.jl'.format(get_fname(url))
    
    try:
        with open(os.path.join(QUOTES_DIR,fname),'r') as f:
            data = f.readlines()
        quotes = []
        for x in data:
            res = json.loads(x.strip())
            res.update({'source':url})                
            quotes.append(res)
            
        return quotes
    except FileNotFoundError:
        print(fname)
        missing.append(fname)

In [316]:
#len(os.listdir(DIR_WITH_NAMES))+len(os.listdir(DIR_WITH_NOS)),len(df)

In [319]:
missing = []
all_quotes = [get_quotes_from_file(url_ix,df,missing) for url_ix in range(0,17621)]
all_quotes = [item for sublist in all_quotes for item in sublist]

In [320]:
len(missing),len(all_quotes)

(0, 336997)

In [321]:
pickle.dump(all_quotes,open('all_quotes.pkl','wb'))

In [198]:
def get_quotes_with_lemma(lemma,element,quote_list):
    return [q for q in quote_list if q[element] is not None and lemmatizer.lemmatize(q[element].lower()) == lemma]

def get_quotes_with_stem(stem_set,element,quote_list):
    return [q for q in quote_list if q[element] is not None and ps.stem(q[element].lower()) in stem_set]

def get_quotes_without_stem(stem_set,element,quote_list):
    return [q for q in quote_list if q[element] is not None and ps.stem(q[element].lower()) not in stem_set]

## Filter to true indirect statements

In [178]:
# Remove quotes with question verb

QUESTION_WORDS = set(['who','what','when','where','why','how','if','whose','whether','which','whom','whence'])
QUESTION_VERBS = set(['ask','wonder','figure','guess','inquire','interrogate','question'])

def has_indirect_question(quote_obj):
    quote = quote_obj['quote']
    first_word = quote[0].lower()
    return first_word in QUESTION_WORDS

def has_question_verb(quote_obj):
    verb = ps.stem(quote_obj['main verb'].lower())
    return verb in QUESTION_VERBS

In [179]:
all_quotes = [q for q in all_quotes if not has_question_verb(q)]
len(all_quotes)

549680

In [180]:
# Set aside quotes with 'be' as main verb

BE_STEMS = set(['is','wa','’s','are',"'s",'be','’m','am','are','’re',"'m","'re","been",'were'])

quotes_without_be = get_quotes_without_stem(BE_STEMS,all_quotes)
len(quotes_without_be)

548113

In [ ]:
#554136

In [181]:
# Remove quotes with 'point','turn' but not 'point_out','turn_out' as main verb

point_quotes = get_quotes_with_stem(['point','turn'],quotes_without_be)
point_out_quotes = [x for x in point_quotes if 'out' in x['verb tokens']]

true_point_out_quotes = []
false_point_out_quotes = []
for x in point_out_quotes:
    indices_out = [i for i,t in enumerate(x['verb tokens']) if t.lower() == 'out']
    prev_toks = [ps.stem(x['verb tokens'][i-1].lower()) for i in indices_out]
    if 'point' in prev_toks or 'turn' in prev_toks:
        true_point_out_quotes.append(x)
    else:
        false_point_out_quotes.append(x)
        
len(true_point_out_quotes),len(false_point_out_quotes)

(2585, 4)

In [ ]:
#(2051, 4)

In [80]:
false_point_out_quotes

[{'quote': ['I',
   'am',
   'going',
   'to',
   'ask',
   'that',
   'you',
   'look',
   'at',
   'him'],
  'verb tokens': ['point', 'him', 'out'],
  'main verb': 'point',
  'subject tokens': None,
  'main subject': None,
  'neg tokens': None,
  'main neg': None,
  'is neg': None,
  'source': 'rssfeeds.usatoday.com/~/613545646/0/news-opinion~A-he-said-she-said-or-a-violent-rape-What-its-like-to-testify-against-your-rapist/'},
 {'quote': ['”',
   'Kappenberger',
   'said',
   'of',
   'the',
   'Environmental',
   'Protection',
   'Agency',
   '’s'],
  'verb tokens': ['has', 'pointed', 'this', 'information', 'out', 'to', 'me'],
  'main verb': 'pointed',
  'subject tokens': ['anyone'],
  'main subject': 'anyone',
  'neg tokens': None,
  'main neg': None,
  'is neg': None,
  'source': 'dailycaller.com/2015/11/18/undetectable-the-impact-a-un-treaty-will-have-on-global-warming/'},
 {'quote': ['signature', 'global', 'warming', 'rule'],
  'verb tokens': ['has', 'pointed', 'this', 'informat

In [182]:
all_quotes = [q for q in quotes_without_be if q not in false_point_out_quotes]
len(all_quotes)

548109

In [ ]:
#554132

In [183]:
# Remove quotes with a set of non-attributive verbs introducing comp clause
OTHER_VERBS = set(['make','made','cause','caus','ensur','like','love',
                    'help','mean','meant','get','got','let','allow','want','becam',
                    'becom','go','goe','lead','have','had','ha',
                  'remain','start','give','work','creat','happen','now',
                  'try','tri','matter','set','produc','do','did','need',
                  'sure','clear','right','requir','use','hold','watch',
                  'take','keep','chang','rule','come','look','put'])

non_other_verb_quotes = get_quotes_without_stem(OTHER_VERBS,all_quotes)
len(non_other_verb_quotes)

442865

In [ ]:
#477533

In [184]:
# Remove quotes with 'seem', 'appear' as main verb

non_seem_quotes = get_quotes_without_stem(['seem','appear'],non_other_verb_quotes)
len(non_seem_quotes)#+len(seem_quotes),len(all_quotes)

440677

In [ ]:
#475345

In [185]:
# Remove verb with non-alpha main verb

non_punc_main_verb_quotes = [q for q in non_seem_quotes if q['main verb'][0].lower().isalpha()]
len(non_punc_main_verb_quotes)

436611

In [ ]:
#471279

In [85]:
non_punc_main_verb_quotes[32432]

{'quote': ['Proof',
  'of',
  'the',
  'experiment',
  "'s",
  'success',
  'is',
  'that',
  'the',
  'customer',
  'will',
  'pay',
  'for',
  'the',
  'technology',
  'and',
  'start',
  'using',
  'the',
  'technology'],
 'verb tokens': ['said'],
 'main verb': 'said',
 'subject tokens': ['Rossi'],
 'main subject': 'Rossi',
 'neg tokens': None,
 'main neg': None,
 'is neg': None,
 'source': 'www.foxnews.com/science/cold-fusion-experiment-major-success-or-complex-hoax'}

In [186]:
pickle.dump(non_punc_main_verb_quotes,open('filtered_quotes.pkl','wb'))

## Examine distribution of indirect statements

In [ ]:
# most common main verbs, subjects, amount of negation, sources

In [187]:
quotes = pickle.load(open('filtered_quotes.pkl','rb'))
len(quotes)

436611

In [188]:
all_main_verb_stems = [ps.stem(x['main verb']).lower() for x in quotes]
counted_main_verb_stems = Counter(all_main_verb_stems)
sorted(counted_main_verb_stems.items(),key=lambda x:x[1],reverse=True)

[('said', 90224),
 ('say', 45485),
 ('think', 18085),
 ('told', 12025),
 ('know', 11974),
 ('believ', 10025),
 ('show', 9118),
 ('found', 8970),
 ('suggest', 8126),
 ('see', 7116),
 ('note', 7020),
 ('argu', 6745),
 ('report', 5441),
 ('ad', 5076),
 ('claim', 4496),
 ('explain', 4369),
 ('tell', 4357),
 ('wrote', 4301),
 ('find', 3671),
 ('warn', 3320),
 ('announc', 3247),
 ('hope', 3192),
 ('estim', 3124),
 ('conclud', 2965),
 ('understand', 2665),
 ('expect', 2664),
 ('thought', 2657),
 ('consid', 2344),
 ('agre', 2320),
 ('point', 2239),
 ('predict', 2175),
 ('state', 2088),
 ('indic', 2020),
 ('declar', 1983),
 ('write', 1920),
 ('assum', 1880),
 ('acknowledg', 1877),
 ('realiz', 1823),
 ('determin', 1790),
 ('insist', 1751),
 ('decid', 1554),
 ('knew', 1547),
 ('admit', 1445),
 ('reveal', 1365),
 ('feel', 1269),
 ('learn', 1206),
 ('worri', 1175),
 ('rememb', 1171),
 ('add', 1125),
 ('saw', 1071),
 ('confirm', 1059),
 ('imagin', 1055),
 ('prove', 1030),
 ('seen', 1022),
 ('fear', 

In [189]:
'wonder' in counted_main_verb_stems

False

In [176]:
other_quotes = get_quotes_with_stem({
    'put'
},quotes)

In [177]:
other_quotes[:10]

[{'quote': ['we', 'heads', 'down'],
  'verb tokens': ['are', 'going', 'to', 'put'],
  'main verb': 'put',
  'subject tokens': ['we'],
  'main subject': 'we',
  'neg tokens': None,
  'main neg': None,
  'is neg': None,
  'source': 'www.thenation.com/article/archive/trump-climate-whistleblowers/'},
 {'quote': ['Humans', 'played', 'an', 'important', 'role', ',', 'too'],
  'verb tokens': ['put',
   'pressure',
   'on',
   'the',
   'animals',
   'through',
   'hunting',
   'scattering',
   'the',
   'megafauna',
   'throughout',
   'the',
   'region'],
  'main verb': 'put',
  'subject tokens': ['Humans', 'presence'],
  'main subject': 'presence',
  'neg tokens': None,
  'main neg': None,
  'is neg': None,
  'source': 'www.nytimes.com/2016/06/18/science/patagonia-extinctions-global-warming.html'},
 {'quote': ['calls', 'online'],
  'verb tokens': ['then', 'puts', 'the', 'recordings', 'of', 'Conference'],
  'main verb': 'puts',
  'subject tokens': None,
  'main subject': None,
  'neg tokens':

In [ ]:
# 

A lot of quotes don't have a subject:

In [190]:
len([q for q in quotes if q['main subject'] is None])

44350

The most common subject stems--it looks like coref didn't fully do its job.

In [191]:
all_main_subj_stems = [ps.stem(x['main subject']).lower() for x in quotes if x['main subject'] is not None]
counted_main_subj_stems = Counter(all_main_subj_stems)
sorted(counted_main_subj_stems.items(),key=lambda x:x[1],reverse=True)

[('i', 27014),
 ('we', 14231),
 ('you', 10152),
 ('it', 8506),
 ('scientist', 6650),
 ('he', 5808),
 ('studi', 5787),
 ('research', 5183),
 ('report', 4995),
 ('peopl', 4212),
 ('trump', 3895),
 ('offici', 3570),
 ('’s', 3097),
 ('she', 2845),
 ('expert', 2798),
 ('they', 2221),
 ('obama', 2030),
 ('one', 1982),
 ('author', 1872),
 ('group', 1818),
 ('that', 1750),
 ('percent', 1598),
 ('agenc', 1481),
 ('some', 1335),
 ('compani', 1315),
 ('sander', 1276),
 ('thi', 1247),
 ('administr', 1189),
 ('govern', 1128),
 ('“', 956),
 ('team', 954),
 ('critic', 952),
 ('poll', 933),
 ('democrat', 918),
 ('presid', 899),
 ('data', 892),
 ('pruitt', 869),
 ('those', 861),
 ('mani', 855),
 ('analysi', 844),
 ('depart', 838),
 ('leader', 824),
 ('us', 824),
 ('biden', 811),
 ('activist', 781),
 ('epa', 779),
 ('state', 774),
 ('hi', 768),
 ('brown', 752),
 ('organ', 747),
 ('other', 746),
 ('environmentalist', 741),
 ('time', 735),
 ('paper', 730),
 ('analyst', 725),
 ('republican', 723),
 ('panel

In [201]:
he_quotes = get_quotes_with_stem({'he'},'main subject',quotes)
len(he_quotes)

5808

In [202]:
he_quotes[0]

{'quote': ['People',
  'free',
  '-',
  'ride',
  'when',
  'People',
  'jump',
  'the',
  'turnstile',
  'on',
  'the',
  'subway'],
 'verb tokens': ['said'],
 'main verb': 'said',
 'subject tokens': ['he'],
 'main subject': 'he',
 'neg tokens': None,
 'main neg': None,
 'is neg': None,
 'source': 'www.washingtonpost.com/national/health-science/a-kind-of-dark-realism-why-the-climate-change-problem-is-starting-to-look-too-big-to-solve/2018/12/03/378e49e4-e75d-11e8-a939-9469f1166f9d_story.html'}

Somewhere between 8.3k to 8.8k quotes have a negated main verb.

In [192]:
len([q for q in quotes if q['is neg']])

8350

In [193]:
len([q for q in quotes if q['main neg'] is not None])

8804

In [194]:
all_quote_domains = [df.loc[df.url==q['source']]['pretty_domain'].values[0] for q in quotes]
counted_quote_domains = Counter(all_quote_domains)
sorted(counted_quote_domains.items(),key=lambda x:x[1],reverse=True)

[('New York Times', 114096),
 ('Mother Jones', 43927),
 ('Fox', 34783),
 ('Christian Science Monitor', 29316),
 ('Washington Post', 29108),
 ('The Nation', 22924),
 ('Breitbart', 22808),
 ('Guardian (US)', 22716),
 ('Vox', 18416),
 ('Buzzfeed', 9226),
 ('Daily Caller', 8395),
 ('Grist', 8242),
 ("Children's Health Defense", 7671),
 ('Shot of Prevention', 7028),
 ('Blaze', 5930),
 ('Alternet', 5925),
 ('National Review', 5090),
 ('PJ Media', 4943),
 ('Drudgereport', 3754),
 ('USA Today', 3176),
 ('Reason', 2384),
 ('Newsweek', 2241),
 ('Quartz', 2180),
 ('The Progressive', 1849),
 ('Voices for Vaccines', 1752),
 ('The Verge', 1716),
 ('Democracy Now', 1426),
 ('American Thinker', 1413),
 ('NBC', 1342),
 ('CNS', 1336),
 ('Sgtreport', 1095),
 ('New York Magazine', 1083),
 ('Redstate', 1081),
 ('Activist Post', 1073),
 ('Vice', 1064),
 ('Infowars', 777),
 ('In These Times', 743),
 ('News With Views', 616),
 ('Citizens', 586),
 ('Hot Air', 505),
 ('The Week', 472),
 ('The American Spectator

# Additional pre-processing

## Add lemma, stem info

## Add NER tags?